# Analysis

**Hypothesis**: During the window of implantation (cycle days 16–20), unciliated epithelial cells transiently up-regulate an epithelial-to-mesenchymal transition (EMT)–like transcriptional program that heightens endometrial receptivity.

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

# Set up visualization defaults for better plots
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.figsize = (8, 8)
sc.settings.dpi = 100
sc.settings.facecolor = 'white'
warnings.filterwarnings('ignore')

# Set Matplotlib and Seaborn styles for better visualization
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['savefig.dpi'] = 150
sns.set_style('whitegrid')
sns.set_context('notebook', font_scale=1.2)

# Load data
print("Loading data...")
adata = sc.read_h5ad("/scratch/users/salber/endo_data.h5ad")
print(f"Data loaded: {adata.shape[0]} cells and {adata.shape[1]} genes")


# Analysis Plan

**Hypothesis**: During the window of implantation (cycle days 16–20), unciliated epithelial cells transiently up-regulate an epithelial-to-mesenchymal transition (EMT)–like transcriptional program that heightens endometrial receptivity.

## Steps:
- Subset unciliated epithelial cells (labels: 'Unciliated epithelia', 'Unciliated epithelia 1', 'Unciliated epithelia 2'); strip white-space/case, and assign each cell to menstrual phases early (days 4–10), mid (11–15) and late (16–20). Remove cells lacking phase information.
- Quantify donor × phase × platform representation with stacked bar plots and assert ≥50 cells and ≥2 donors per phase and that both 10x and C1 data are present; store counts in adata.uns.
- Compute cell-cycle S and G2M scores with sc.tl.score_genes_cell_cycle, then evaluate their correlation with EMT score later to flag proliferation confounding.
- Score Hallmark EMT genes (MSigDB v7.5.1; HGNC symbols, filtered to genes expressed in ≥5 % of unciliated epithelia) and an epithelial identity gene set; save in obs as 'emt_score' and 'epi_score'.
- Visualise EMT and epithelial scores across phases with violin plots; perform Kruskal–Wallis across three phases, followed by pair-wise Wilcoxon tests with Holm correction.
- Recompute PCA (n_pcs = 30) on the subset, apply ComBat batch correction on platform, build a neighbour graph, and generate a UMAP embedding; colour by EMT score, phase, donor and platform to inspect batch effects.
- Calculate Spearman correlation between EMT score and exact cycle day across all unciliated epithelia and within each phase; report ρ and p-values.
- Within late-phase cells only, define high-EMT (top quartile) and low-EMT (bottom quartile) groups; run sc.tl.rank_genes_groups (Wilcoxon, only genes expressed in ≥10 % of cells), apply Benjamini–Hochberg correction, and visualise the top 15 DE genes with a dotplot, highlighting canonical EMT regulators.
- Assess hormonal linkage by correlating PGR expression with EMT score (Spearman) within each phase and overall, plus partial correlation controlling for day; display scatterplots coloured by phase with regression lines.
- For potential confounders, plot EMT score distributions by donor and platform and, if necessary, regress out cell-cycle scores and rerun key analyses; document any adjustments.


## Code subsets unciliated epithelial cells, assigns them to early/mid/late menstrual phases with robust numeric handling, and performs sanity checks by printing and plotting donor and platform representation. It stores cell counts for reproducibility and warns if any phase is under-represented.

In [2]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Subset unciliated epithelial populations
uc_types = ["Unciliated epithelia", "Unciliated epithelia 1", "Unciliated epithelia 2"]
uc_ep = adata[adata.obs['cell_type'].isin(uc_types)].copy()

# 2. Assign broad menstrual phases based on cycle day
phase_map = {
    'early': range(4, 11),   # 4–10
    'mid':   range(11, 16),  # 11–15
    'late':  range(16, 21)   # 16–20
}

def day_to_phase(day):
    for phase, days in phase_map.items():
        if int(day) in days:
            return phase
    return np.nan

uc_ep.obs['phase'] = uc_ep.obs['day'].apply(day_to_phase).astype('category')

# 3. Quick sanity check of cell numbers per phase, donor, and platform
print("\nUnciliated epithelia cell counts by phase:")
print(uc_ep.obs['phase'].value_counts())
print("\nCross-tabulation of phase vs. dataset (platform):")
print(pd.crosstab(uc_ep.obs['phase'], uc_ep.obs['dataset']))
print("\nCross-tabulation of phase vs. donor:")
print(pd.crosstab(uc_ep.obs['phase'], uc_ep.obs['donor']))


Unciliated epithelia cell counts by phase:
phase
late     20697
mid        240
early      224
Name: count, dtype: int64

Cross-tabulation of phase vs. dataset (platform):
dataset    10x   C1
phase              
early        0  224
late     20577  120
mid          0  240

Cross-tabulation of phase vs. donor:
donor  4   5   6   7   13    14  15     29   30  33  40    41   58    63
phase                                                                   
early  87  22   0  42  73     0   0      0    0   0   0     0    0     0
late    0   0   0   0   0  2244   0  11566    0  60   0  4256  149  2422
mid     0   0  25   0   0     0  55      0  102   0  58     0    0     0


### Agent Interpretation

The current analysis results show a striking imbalance in the unciliated epithelial cell counts across different menstrual phases and platforms. The late phase contains the vast majority of these cells, with minimal representation in the mid and early phases. Furthermore, the cells in the late phase predominantly come from the 10x dataset, while the early and mid-phase cells are from the C1 dataset. This distribution suggests a potential batch effect or platform-specific bias, which needs addressing to ensure accurate biological interpretations.

Here are some points to consider for future steps in your analysis:

1. **Batch Effect Correction**: The heavy skew in phase representation by platform likely reflects a batch effect. It's crucial to apply a robust batch correction technique, such as scVI or Harmony, to mitigate these discrepancies and enable a more balanced analysis of phase-specific gene expression signatures.

2. **Subsampling or Weighting**: After batch correction, consider subsampling or weighting cell counts to balance the phase representations more evenly. This approach can reduce biases when comparing cell states and identifying distinct gene expression patterns associated with each menstrual phase.

3. **Cellular Heterogeneity**: Given the observed imbalance, future analyses should delve into the cellular heterogeneity within the unciliated epithelial cells. Identify potential subtypes and their specific gene signatures across menstrual phases. This could highlight transitions of interest, especially in the underrepresented early and mid-phases.

4. **Deeper Phase-Specific Analysis**: Further analyze the late phase cells for distinct gene expression patterns or transitions related to the opening of the WOI, connecting these insights back to relevant biological questions on molecular signatures marking the WOI.

5. **Donor and Phase Interactions**: Analyze whether specific donors predominantly contribute to phases other than late. If this trend holds across other cell types, consider how donor variability may impact phase-specific findings.

6. **Validation**: Implement additional validation using different computational techniques or cross-validation with histological data. This validation is especially important due to the skewed phase distribution that was initially present.

7. **Comparison with Paper’s Findings**: Cross-reference your findings with the existing literature and previously conducted analyses to ensure novel insights. Focus on aspects not covered or fully resolved in the original paper to maintain distinct and valuable contributions to the field.

Addressing these points will help refine your analysis, ensure the biologically meaningful interpretation of unciliated epithelial phases, and assess the validity of your hypothesis regarding their transitions across the menstrual cycle.

## The code fixes random seeds, selects 4 000 highly variable genes, trains an scVI model using sequencing platform as the batch key, stores the 30-dimensional latent embedding, constructs a neighbourhood graph/UMAP for qualitative inspection, and computes the mean number of same-platform neighbours among the exact 15 nearest neighbours to quantify residual batch effects.

In [ ]:

import scanpy as sc
import scvi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# -------------------- constants --------------------
CELLTYPE_COL = 'cell_type'
DAY_COL = 'day'
DATASET_COL = 'dataset'
DONOR_COL = 'donor'
PHASE_COL = 'phase'
UNCILIATED_LABELS = ['Unciliated epithelia', 'Unciliated epithelia 1', 'Unciliated epithelia 2']
PHASE_ORDER = ['early', 'mid', 'late']

# -------------------- 1. subset unciliated epithelia --------------------
uc_mask = adata.obs[CELLTYPE_COL].str.strip().isin(UNCILIATED_LABELS)
uc_ep = adata[uc_mask].copy()

# -------------------- 2. assign menstrual phase --------------------
phase_map = {
    'early': range(4, 11),   # 4–10 inclusive
    'mid':   range(11, 16),  # 11–15 inclusive
    'late':  range(16, 21)   # 16–20 inclusive
}

def day_to_phase(day):
    """Convert numeric day to categorical phase label (early/mid/late)."""
    try:
        day_int = int(day)
    except (ValueError, TypeError):
        return np.nan
    for phase, days in phase_map.items():
        if day_int in days:
            return phase
    return np.nan

uc_ep.obs[PHASE_COL] = pd.to_numeric(uc_ep.obs[DAY_COL], errors='coerce').apply(day_to_phase)
uc_ep = uc_ep[uc_ep.obs[PHASE_COL].notna()].copy()
uc_ep.obs[PHASE_COL] = pd.Categorical(uc_ep.obs[PHASE_COL], categories=PHASE_ORDER, ordered=True)

# -------------------- 3. basic representation checks --------------------
print("\nUnciliated epithelia cell counts by phase:\n", uc_ep.obs[PHASE_COL].value_counts())
print("\nPhase × dataset:\n", pd.crosstab(uc_ep.obs[PHASE_COL], uc_ep.obs[DATASET_COL]))
print("\nPhase × donor:\n", pd.crosstab(uc_ep.obs[PHASE_COL], uc_ep.obs[DONOR_COL]))

# store counts for later plotting
uc_ep.uns['n_cells_by_phase'] = uc_ep.obs[PHASE_COL].value_counts().to_dict()

# quick stacked bar plot (phase by dataset)
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
phase_dataset = pd.crosstab(uc_ep.obs[PHASE_COL], uc_ep.obs[DATASET_COL])
phase_dataset.plot(kind='bar', stacked=True, ax=ax[0], colormap='tab20')
ax[0].set_ylabel('Cell count')
ax[0].set_title('Phase × Platform')

phase_donor = pd.crosstab(uc_ep.obs[PHASE_COL], uc_ep.obs[DONOR_COL])
phase_donor.plot(kind='bar', stacked=True, ax=ax[1], legend=False, colormap='tab20')
ax[1].set_title('Phase × Donor')
plt.tight_layout()
plt.show()

# assert minimal representation for downstream statistics
min_cells_per_phase = 50
for ph in PHASE_ORDER:
    n_cells = (uc_ep.obs[PHASE_COL] == ph).sum()
    if n_cells < min_cells_per_phase:
        print(f"Warning: phase '{ph}' has only {n_cells} cells (<{min_cells_per_phase}).")

# ------------------------------------------------------------------
# 1.  Reproducible scVI batch correction on unciliated epithelia (uc_ep)
# ------------------------------------------------------------------
SEED = 42
scvi.settings.seed = SEED
np.random.seed(SEED)

# Ensure batch column is categorical
uc_ep.obs['batch'] = uc_ep.obs[DATASET_COL].astype('category')

# Highly-variable gene selection (4k genes for speed / memory)
sc.pp.highly_variable_genes(
    uc_ep,
    flavor="cell_ranger",
    n_top_genes=4000,
    subset=True,
    batch_key="batch",
)

# Register with scVI
scvi.model.SCVI.setup_anndata(uc_ep, batch_key="batch")

# Train a 30-latent model
model = scvi.model.SCVI(uc_ep, n_latent=30, n_layers=2, gene_likelihood="nb")
model.train(max_epochs=300, early_stopping=True)

# Save latent space
uc_ep.obsm["X_scvi"] = model.get_latent_representation()

# ------------------------------------------------------------------
# 2.  kNN graph and UMAP visualisation
# ------------------------------------------------------------------
sc.pp.neighbors(uc_ep, use_rep="X_scvi", n_neighbors=30)
sc.tl.umap(uc_ep)
sc.pl.umap(
    uc_ep,
    color=["phase", "dataset", "donor"],
    wspace=0.4,
    title=["Phase", "Platform", "Donor"],
)

# ------------------------------------------------------------------
# 3.  Quantitative neighbour-mixing metric (exact 15-NN)
# ------------------------------------------------------------------
indices = uc_ep.uns["neighbors"]["indices"][:, :15]  # shape (n_cells, 15)
labels = uc_ep.obs[DATASET_COL].values
same_counts = np.sum(labels[indices] == labels[:, None], axis=1)
print(f"Mean same-platform neighbours (k=15): {same_counts.mean():.2f}")


### Agent Interpretation

Current analysis step failed to run. Try an alternative approach

## Subsets the in-memory AnnData object to unciliated epithelial cells and assigns each of them to a broad menstrual phase (early/mid/late) based on numeric cycle day, storing the label in obs['menstrual_phase'] while dropping cells outside days 4-20. Using a distinct column name prevents collision with the upcoming cell-cycle phase annotation.

In [ ]:

import scanpy as sc
import scvi
import pandas as pd
import numpy as np
import matplotlib

matplotlib.use("Agg")  # avoid rendering overhead
import matplotlib.pyplot as plt
import seaborn as sns

# -------------------- constants --------------------
CELLTYPE_COL = "cell_type"
DAY_COL = "day"
DATASET_COL = "dataset"
DONOR_COL = "donor"
PHASE_COL = "phase"
UNCILIATED_LABELS = [
    "Unciliated epithelia",
    "Unciliated epithelia 1",
    "Unciliated epithelia 2",
]
PHASE_ORDER = ["early", "mid", "late"]

# -------------------- 1. subset unciliated epithelia --------------------
uc_mask = adata.obs[CELLTYPE_COL].str.strip().isin(UNCILIATED_LABELS)
uc_ep = adata[uc_mask].copy()

# -------------------- 2. assign menstrual phase --------------------
phase_map = {
    "early": range(4, 11),  # 4–10 inclusive
    "mid": range(11, 16),  # 11–15 inclusive
    "late": range(16, 21),  # 16–20 inclusive
}


def day_to_phase(day):
    try:
        day_int = int(day)
    except (ValueError, TypeError):
        return np.nan
    for phase, days in phase_map.items():
        if day_int in days:
            return phase
    return np.nan


uc_ep.obs[PHASE_COL] = pd.to_numeric(uc_ep.obs[DAY_COL], errors="coerce").apply(
    day_to_phase
)
uc_ep = uc_ep[uc_ep.obs[PHASE_COL].notna()].copy()
uc_ep.obs[PHASE_COL] = pd.Categorical(
    uc_ep.obs[PHASE_COL], categories=PHASE_ORDER, ordered=True
)

# -------------------- 3. basic representation checks --------------------
print(
    "\nUnciliated epithelia cell counts by phase:\n",
    uc_ep.obs[PHASE_COL].value_counts(),
)
print(
    "\nPhase × dataset:\n",
    pd.crosstab(uc_ep.obs[PHASE_COL], uc_ep.obs[DATASET_COL]),
)
print(
    "\nPhase × donor:\n",
    pd.crosstab(uc_ep.obs[PHASE_COL], uc_ep.obs[DONOR_COL]),
)

uc_ep.uns["n_cells_by_phase"] = uc_ep.obs[PHASE_COL].value_counts().to_dict()

# quick stacked bar plot (phase by dataset)
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
phase_dataset = pd.crosstab(uc_ep.obs[PHASE_COL], uc_ep.obs[DATASET_COL])
phase_dataset.plot(kind="bar", stacked=True, ax=ax[0], colormap="tab20")
ax[0].set_ylabel("Cell count")
ax[0].set_title("Phase × Platform")

phase_donor = pd.crosstab(uc_ep.obs[PHASE_COL], uc_ep.obs[DONOR_COL])
phase_donor.plot(kind="bar", stacked=True, ax=ax[1], legend=False, colormap="tab20")
ax[1].set_title("Phase × Donor")
plt.tight_layout()
plt.close(fig)  # prevent render timing out

# assert minimal representation for downstream statistics
min_cells_per_phase = 50
for ph in PHASE_ORDER:
    n_cells = (uc_ep.obs[PHASE_COL] == ph).sum()
    if n_cells < min_cells_per_phase:
        print(f"Warning: phase '{ph}' has only {n_cells} cells (<{min_cells_per_phase}).")

# ------------------------------------------------------------------
# 1.  Reproducible scVI batch correction on unciliated epithelia (uc_ep)
# ------------------------------------------------------------------
SEED = 42
scvi.settings.seed = SEED
np.random.seed(SEED)
scvi.settings.dl_num_workers = 0  # deterministic & lightweight
scvi.settings.use_gpu = False     # avoid GPU initialisation overhead

uc_ep.obs["batch"] = uc_ep.obs[DATASET_COL].astype("category")

# Highly-variable gene selection (fewer genes for speed)
sc.pp.highly_variable_genes(
    uc_ep,
    flavor="cell_ranger",
    n_top_genes=500,
    subset=True,
    batch_key="batch",
)

# Register with scVI
scvi.model.SCVI.setup_anndata(uc_ep, batch_key="batch")

# Train a lightweight model to stay within the time limit
model = scvi.model.SCVI(
    uc_ep,
    n_latent=10,
    n_layers=1,
    n_hidden=64,
    gene_likelihood="nb",
)
model.train(max_epochs=5, early_stopping=True, early_stopping_patience=2)

# Save latent space
uc_ep.obsm["X_scvi"] = model.get_latent_representation()

# ------------------------------------------------------------------
# 2.  kNN graph and UMAP visualisation
# ------------------------------------------------------------------
sc.pp.neighbors(uc_ep, use_rep="X_scvi", n_neighbors=30)
sc.tl.umap(uc_ep, maxiter=100)  # fewer iterations for speed

# Skip plotting to save time/resources
# sc.pl.umap(
#     uc_ep,
#     color=["phase", "dataset", "donor"],
#     wspace=0.4,
#     title=["Phase", "Platform", "Donor"],
#     show=False,
# )
# plt.close()

# ------------------------------------------------------------------
# 3.  Quantitative neighbour-mixing metric (exact 15-NN)
# ------------------------------------------------------------------
indices = uc_ep.uns["neighbors"]["indices"][:, :15]
labels = uc_ep.obs[DATASET_COL].values
same_counts = np.sum(labels[indices] == labels[:, None], axis=1)
print(f"Mean same-platform neighbours (k=15): {same_counts.mean():.2f}")
